In [1]:

from PIL import Image
import requests
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm
import os
from glob import glob
import pandas as pd
import json
from skimage.draw import polygon2mask
import shutil
def createDirectory(directory):
    """_summary_
        create Directory
    Args:
        directory (string): file_path
    """
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [7]:
class_list=['NT_epithelial',
 'NT_immune',
 'Tumor',]
json_list=glob('../../data/synth_json/_BRDC/유형*/*.json')
image_path="../../data/synth_json/_BRDC/**/"
save_img_path='../../data/area_segmentation/_BRDC/image/'
save_mask_path='../../data/area_segmentation/_BRDC/mask/'

In [8]:
createDirectory(save_img_path)
createDirectory(save_mask_path)

for j in tqdm(range(len(json_list))):
    with open(json_list[j], 'r') as f:
        json_data = json.load(f)
    anno=json_data['content']['file']['objects']
    file_name=json_data['content']['file']['filename']
    # for i in range(len(anno)):
    #     class_list.append(anno[i]['label_nm'])
    # class_list=list(set(class_list))
    ii1=glob(image_path+file_name)[0]
    shutil.copyfile(ii1, save_img_path+file_name)
    mask=np.zeros((1024,1024,len(class_list)+1))
    for k in range(len(anno)):
        
        coordinate=np.array(anno[k]['coordinate'])
        coordinate[:,[0, 1]]=coordinate[:,[1, 0]]
        class_nm=anno[k]['label_nm']
        mask_temp=polygon2mask((1024,1024), coordinate)
        mask[...,class_list.index(class_nm)+1]+=mask_temp
    mask[...,0]=np.where(mask[...,1:].sum(axis=2)>=1,0,1)
    mask=np.where(mask>=1,255,0)
    reversed_index = np.argmax(mask[:,:,::-1], axis=2)
    final_index = len(class_list)-reversed_index
    for i in range(len(class_list)):
        createDirectory(save_mask_path+class_list[i])
        cv2.imwrite(save_mask_path+class_list[i]+'/'+file_name,np.where(final_index==i+1,255,0))
    createDirectory(save_mask_path+'npy/')
    np.save(save_mask_path+'npy/'+file_name.replace('.jpeg','.npy'),mask)

100%|██████████| 215/215 [02:20<00:00,  1.53it/s]


In [5]:
json_list[j]

'../../data/synth_json/_BRDC/유형6/NIA_S_BRDC_6_000008.json'